# Module

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import google.generativeai as gemini
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
import os

# Settings our Model

In [2]:
load_dotenv()
api = os.getenv("google")

gemini.configure(api_key=api)

model = gemini.GenerativeModel("gemini-1.5-flash")

response = model.generate_content("hi im zakariya")


response.text

'Hi Zakariya!  Nice to meet you.  How can I help you today?\n'

# Loading the Csv - Knowledge base

In [9]:
loader = CSVLoader(file_path="data.csv",source_column= "input")

data = loader.load()

RuntimeError: Error loading data.csv

# embeddings setup

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api,task_type="retrieval_query")

## Text chunks into embeddings

In [ ]:
text_data = [doc.page_content for doc in data]

vectors = [embeddings.embed_query(text) for text in text_data]

# Vector Database


In [ ]:
vectordb = Chroma.from_texts(
    texts=text_data,  
    embedding=embeddings,
    persist_directory="rova_db"  
)

# Cosine Similarity search

In [ ]:
query = input("enter the question")
search_results = vectordb.similarity_search(query, k=3)

print(search_results)

[Document(metadata={}, page_content='Category: Robotics\nTopic: Basics\nSubtopic: Basics\nInput: Tell me about basics in robotics\nResponse: Robotics is so cool! Start with sensors—they’re like the robot’s senses. Want to learn how I teach them?\nAdditional Notes: Straightforward and conversational tone for direct engagement.'), Document(metadata={}, page_content='Category: Robotics\nTopic: Basics\nSubtopic: Basics\nInput: Tell me about basics in robotics\nResponse: Robotics is so cool! Start with sensors—they’re like the robot’s senses. Want to learn how I teach them?\nAdditional Notes: Straightforward and conversational tone for direct engagement.'), Document(metadata={}, page_content='Category: Robotics\nTopic: Basics\nSubtopic: Basics\nInput: Tell me about basics in robotics\nResponse: Robotics is so cool! Start with sensors—they’re like the robot’s senses. Want to learn how I teach them?\nAdditional Notes: Straightforward and conversational tone for direct engagement.')]


# Combine search results as context

In [ ]:
combined_context = "\n\n".join([res.page_content for res in search_results])

#### make it a single prompt to avoid typeerror

In [ ]:
final_prompt = f"User's Query: {query}\n\nContext:\n{combined_context}\n\nProvide a concise, human-like response based on the context above."

# Directly pass the query and context to the model


In [ ]:
response = model.generate_content(
   final_prompt
)



# Extract generated text


In [ ]:
generated_text = response.candidates[0].content.parts[0].text

print("Generated Text:", generated_text)

Generated Text: Hey! Robotics is awesome!  Let's start with the basics:  sensors are like a robot's eyes and ears.  Want to know more about how they work?

